<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span><ul class="toc-item"><li><span><a href="#Взвешивание-классов" data-toc-modified-id="Взвешивание-классов-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Взвешивание классов</a></span></li><li><span><a href="#Увеличение/уменьшение-выборки" data-toc-modified-id="Увеличение/уменьшение-выборки-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Увеличение/уменьшение выборки</a></span><ul class="toc-item"><li><span><a href="#Увеличение-выборки" data-toc-modified-id="Увеличение-выборки-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Увеличение выборки</a></span></li><li><span><a href="#Уменьшение-выборки" data-toc-modified-id="Уменьшение-выборки-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Уменьшение выборки</a></span></li></ul></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_curve, roc_auc_score, precision_score, recall_score
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder

In [71]:
RANDOM_STATE = 42
# Запускать ли обучение и подбор гиперпараметров для модели
TRAIN_TREE = False
TRAIN_FOREST = True # Полбор гиперпараметров занимает около 8 минут на один прогон. В проекте есть 3 прогона
TRAIN_LOG_REGRESSION = False

In [64]:
data = pd.read_csv('./datasets/Churn.csv')
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


В столбце `Tenure — сколько лет человек является клиентом банка` присутствуют пропущенные значения. Запослним пропуски медианным значением для каждой группы клиентов. Группы сформируем по целевому признаку `Exited`

In [65]:
for t in data['Exited'].unique():
    data.loc[(data['Exited'] == t) & (data['Tenure'].isna()), 'Tenure'] = \
    data.loc[data['Exited'] == t, 'Tenure'].median()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Столбцы:
* `RowNumber — индекс строки в данных`  
* `CustomerId — уникальный идентификатор клиента`  
* `Surname — фамилия`  
не несут полезной информации для классификации. Поэтому удалим данные столбцы из выборки

In [66]:
data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

Закодируем категориальные столбцы методом прямого кодирования

Выделим в отдельные переменные набор признаков и целевой признак

In [67]:
features = data.drop('Exited', axis=1)
target = data['Exited']

Выделим тестовую выборку

In [68]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=.2, stratify=target, random_state=RANDOM_STATE
)

Выделим валидационную выборку

In [50]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train, target_train, test_size=.25, stratify=target_train, random_state=RANDOM_STATE
)

In [51]:
print('Размер тренировочной выборки - ', features_train.shape[0])
print('Размер валидационной выборки - ', features_valid.shape[0])
print('Размер тестовой выборки - ', features_test.shape[0])

Размер тренировочной выборки -  6000
Размер валидационной выборки -  2000
Размер тестовой выборки -  2000


Закодируем категориальные столбцы методом прямого кодирования

In [52]:
def ohe(df, encoder, columns):
    encoder_categorical = pd.DataFrame(encoder.transform(df[columns]).toarray(), index=df.index)
    df = df.join(encoder_categorical)
    df.drop(columns, inplace=True, axis=1)
    return df

In [53]:
categorical = ['Geography', 'Gender']
encoder = OneHotEncoder(drop='first') #handle_unknown='ignore'
encoder.fit(features_train[categorical])

OneHotEncoder(drop='first')

In [54]:
features_train = ohe(features_train, encoder, categorical)
features_valid = ohe(features_valid, encoder, categorical)
features_test = ohe(features_test, encoder, categorical)

In [55]:
features_train.info()
features_valid.info()
features_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6000 entries, 1995 to 1215
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      6000 non-null   int64  
 1   Age              6000 non-null   int64  
 2   Tenure           6000 non-null   float64
 3   Balance          6000 non-null   float64
 4   NumOfProducts    6000 non-null   int64  
 5   HasCrCard        6000 non-null   int64  
 6   IsActiveMember   6000 non-null   int64  
 7   EstimatedSalary  6000 non-null   float64
 8   0                6000 non-null   float64
 9   1                6000 non-null   float64
 10  2                6000 non-null   float64
dtypes: float64(6), int64(5)
memory usage: 691.5 KB
<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, 6263 to 1630
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      2000 non-null   int64  
 1

Выполним стандартизацию данных для количественных признаков

In [56]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']

# Стандартизация тренировочной выборки
scaler_train = StandardScaler()
scaler_train.fit(features_train[numeric])
features_train[numeric] = scaler_train.transform(features_train[numeric])

# Стандартизация валидационной выборки
scaler_valid = StandardScaler()
scaler_valid.fit(features_valid[numeric])
features_valid[numeric] = scaler_valid.transform(features_valid[numeric])

# Стандартизация тестовой выборки
scaler_test = StandardScaler()
scaler_test.fit(features_test[numeric])
features_test[numeric] = scaler_test.transform(features_test[numeric])

## Исследование задачи

Посмотрим распределение классов на обучающей выборке

In [57]:
target_train.value_counts()

Exited
0    4777
1    1223
Name: count, dtype: int64

In [58]:
percent_one = target_train[target_train == 1].count() / target_train.shape[0]
print(f'Положительный класс составляет {percent_one:.2%} всей обучающей выборки')

Положительный класс составляет 20.38% всей обучающей выборки


Видно, что в выборке наблюдается дисбаланс классов. Положительный класс составлятет лишь `20.38%` выборки

Исследуем модель решающего дерева

In [59]:
def get_score(model, features, target):
    predicted = model.predict(features)
    f1 = f1_score(target, predicted)
    probabilities_one = model.predict_proba(features)[:, 1]
    roc_auc = roc_auc_score(target, probabilities_one)
    return f1, roc_auc

In [60]:
def train_decision_tree(features_train, target_train, features_valid, target_valid, max_depth, class_weight=None):
    best_depth = 1
    best_f1_score = 0
    best_roc_auce = 0
    best_model = None
    for depth in range(1, max_depth+1):
        model = DecisionTreeClassifier(max_depth=depth, class_weight=class_weight, random_state=RANDOM_STATE)
        model.fit(features_train, target_train)
        f1, roc_auc = get_score(model, features_valid, target_valid)
        if f1 > best_f1_score:
            best_model = model
            best_depth = depth
            best_f1_score = f1
            best_roc_auc = roc_auc
    return best_model, best_depth, best_f1_score, best_roc_auc

In [61]:
best_model_tree = None
if TRAIN_TREE:
    best_model_tree, best_depth, best_f1_score, best_roc_auc = train_decision_tree(
        features_train, target_train, features_valid, target_valid, 50)

    print(f'Best max depth = {best_depth}')
    print(f'Best f1 score = {best_f1_score:.4}')
    print(f'Best roc-auc = {best_roc_auc:.4}')

Best max depth = 2  
Best f1 score = 0.5007  
Best roc-auc = 0.7408

Модель решающего дерева показывает лучший результат при максимальной глубине равной 2. Значение метрики F1 на валидационной выборке составило `0.5007`. Площадь под ROC кривой составила `0.7408`, что является показателем хорошей разделяющей способности алгоритма. Однако делать такие выводы нельзя из-за дисбаланса обучающей выборки

Исследуем модель случайного леса

In [72]:
def train_random_forest(features_train, target_train, features_valid, target_valid, max_depth, n_estimators, class_weight=None):
    best_depth = 1
    best_n_estimators = 1
    best_f1_score = 0
    best_roc_auc = 0
    best_model = None
    for n in range(5, n_estimators+1, 5):
        for depth in range(1, max_depth+1):
            model = RandomForestClassifier(
                n_estimators=n, max_depth=depth, class_weight=class_weight,random_state=RANDOM_STATE, n_jobs=-1)
            model.fit(features_train, target_train)
            f1, roc_auc = get_score(model, features_valid, target_valid)      
            if f1 > best_f1_score:
                best_model = model
                best_f1_score = f1
                best_n_estimators = n
                best_depth = depth
                best_roc_auc = roc_auc
    return best_model, best_depth, best_n_estimators, best_f1_score, best_roc_auc

In [73]:
%%time
best_model_rand_forest = None
if TRAIN_FOREST:
    best_model_rand_forest, best_depth, best_n_estimators, best_f1_score, best_roc_auc = train_random_forest(
        features_train, target_train, features_valid, target_valid, 30, 155)

    print(f'Best max depth = {best_depth}')
    print(f'Best n_estimators = {best_n_estimators}')    
    print(f'Best f1 score = {best_f1_score:.4}')
    print(f'Best roc-auc = {best_roc_auc:.4}')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\a_a_k\AppData\Roaming\Python\Python311\site-packages\IPython\core\magics\execution.py", line 1340, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 3, in <module>
  File "C:\Users\a_a_k\AppData\Local\Temp\ipykernel_21292\4168872328.py", line 11, in train_random_forest
    model.fit(features_train, target_train)
  File "c:\Python311\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\sklearn\ensemble\_forest.py", line 348, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\sklearn\base.py", line 621, in _validate_data
    X, y = check_X_y(X, y, **check_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\sklearn\utils\validation.py", line 1147, in check_X_y
    X = check_array(
   

Best max depth = 22  
Best n_estimators = 45  
Best f1 score = 0.6012  
Best roc-auc = 0.8447

Модель случайного показывает лучший результат при максимальной глубине равной 22 и количестве решателей равным 45. Значение метрики F1 на валидационной выборке составило `0.6012`. Площадь под ROC кривой составила `0.8447`, что является показателем хорошей разделяющей способности алгоритма. Однако делать такие выводы нельзя из-за дисбаланса обучающей выборки

Исследуем модель логистической регрессии

In [24]:
def train_log_regression(features_train, target_train, features_valid, target_valid, max_iter=100, class_weight=None):
    best_c = 0
    best_f1_score = 0
    best_roc_auc = 0
    best_model = None
    for c in np.arange(.1, 5, .1):
        model = LogisticRegression(
            max_iter=max_iter, solver='liblinear', C=c, class_weight=class_weight, random_state=RANDOM_STATE
        )
        model.fit(features_train, target_train)
        f1, roc_auc = get_score(model, features_valid, target_valid)
        if f1 > best_f1_score:
            best_model = model
            best_f1_score = f1
            best_roc_auc = roc_auc
            best_c = c
    return best_model, best_c, best_f1_score, best_roc_auc

In [25]:
model_log_regression = None
if TRAIN_LOG_REGRESSION:
    model_log_regression, best_c, best_f1_score, best_roc_auc = train_log_regression(
        features_train, target_train, features_valid, target_valid)
    print(f'f1 score = {best_f1_score:.4}')
    print(f'roc-auc = {best_roc_auc:.4}')
    print(f'Best C = {best_c:.2}')

f1 score = 0.3201
roc-auc = 0.7556
Best C = 1.2


f1 score = 0.3201  
roc-auc = 0.7556  
Best C = 1.2

Логистическая регрессия обученная на 100 итерациях на валидационной выборке показала значение метрики F1 равное `0.3201`

**Выводы:** В результате исследования моделей на несбалансированной выборке было обучено 3 модели: решающее дерево, случайный лес и логистическая регрессия.  
* Модель решающего дерева показала лучший результат при максимальной глубине равной 2. Значение метрики F1 на валидационной выборке составило `0.5007` .  
* Модель случайного дерева показала лучший результат при максимальной глубине равной 22 и количестве решателей равным 45. Значение метрики F1 на валидационной выборке составило `0.6012` .    
* Модель логистической регрессии, обученная на 100 итерациях, на валидационной выборке показала значение метрики F1 равное `0.3201` .  

Таким образом, лучший результат показала модель случайного леса.

## Борьба с дисбалансом

### Взвешивание классов

Исследуем модель решающего дерева

In [26]:
best_model_tree_w_balanced = None
if TRAIN_TREE:
    best_model_tree_w_balanced, best_depth, best_f1_score, best_roc_auc = train_decision_tree(
        features_train, target_train, features_valid, target_valid, 30, class_weight='balanced')

    print(f'Best max depth = {best_depth}')
    print(f'Best f1 score = {best_f1_score:.4}')
    print(f'Best roc-auc = {best_roc_auc:.4}')

Best max depth = 6
Best f1 score = 0.5875
Best roc-auc = 0.8348


Best max depth = 6  
Best f1 score = 0.5875  
Best roc-auc = 0.8348

Исследуем модель случайного леса

In [27]:
%%time
best_model_rand_forest_w_balanced = None
if TRAIN_FOREST:
    best_model_rand_forest_w_balanced, best_depth, best_n_estimators, best_f1_score, best_roc_auc = train_random_forest(
        features_train, target_train, features_valid, target_valid, 30, 155, class_weight='balanced')

    print(f'Best max depth = {best_depth}')
    print(f'Best n_estimators = {best_n_estimators}')    
    print(f'Best f1 score = {best_f1_score:.4}')
    print(f'Best roc-auc = {best_roc_auc:.4}')

Best max depth = 10
Best n_estimators = 135
Best f1 score = 0.6434
Best roc-auc = 0.8579
CPU times: user 7min 53s, sys: 1.95 s, total: 7min 55s
Wall time: 7min 55s


Best max depth = 10  
Best n_estimators = 135  
Best f1 score = 0.6434  
Best roc-auc = 0.8579

Исследуем модель логистической регрессии

In [28]:
model_log_regr_w_balanced = None
if TRAIN_LOG_REGRESSION:
    model_log_regr_w_balanced, best_c, best_f1_score, best_roc_auc = train_log_regression(
        features_train, target_train, features_valid, target_valid, class_weight='balanced')
    print(f'f1 score = {best_f1_score:.4}')
    print(f'roc-auc = {best_roc_auc:.4}')
    print(f'Best C = {best_c:.2}')

f1 score = 0.481
roc-auc = 0.7602
Best C = 0.2


f1 score = 0.481  
roc-auc = 0.7602  
Best C = 0.2

**Вывод:** Использование взвешенных классов позволило добиться баланса в обучающей выборке. Это позволило увеличить качество моделей.  
* Модель решающего дерева показала лучший результат при максимальной глубине равной 6. Значение метрики F1 на валидационной выборке выросло до `0.5875` . Площадь под ROC кривой для данной модели составила `0.8348`, что говорит о достаточно хорошей разделяющей способности данной модели.  
* Модель случайного леса показала лучший результат при максимальной глубине равной 10 и количестве решателей равным 135. Значение метрики F1 на валидационной выборке выросло до `0.6434` . Площадь под ROC кривой для данной модели составила `0.8579`, что говорит о хорошей разделяющей способности данной модели. Это лучший результат из всех трех моделей.  
* Модель логистической регрессии, обученная на 100 итерациях, на валидационной выборке показала значение метрики F1 выросло до `0.481` . Площадь под ROC кривой для данной модели составила `0.7602`. Разделяющая способность данной модели худшая из всех трех.  

Таким образом, лучший результат показала модель случайного леса и имеет лучшую разделяющую способнось.

### Увеличение/уменьшение выборки

In [29]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_one = features[target == 1]
    target_zeros = target[target == 0]
    target_one = target[target == 1]
    
    features_upsample = pd.concat([features_zeros] + [features_one] * repeat)
    target_upsample = pd.concat([target_zeros] + [target_one] * repeat)
    features_upsample, target_upsample = shuffle(
        features_upsample, target_upsample, random_state=RANDOM_STATE)
    
    return features_upsample, target_upsample

In [30]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_one = features[target == 1]
    target_zeros = target[target == 0]
    target_one = target[target == 1]
    
    features_downsample = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=RANDOM_STATE)] + [features_one])
    target_downsample = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=RANDOM_STATE)] + [target_one])
    features_downsample, target_downsample = shuffle(
        features_downsample, target_downsample, random_state=RANDOM_STATE)
    
    return features_downsample, target_downsample

#### Увеличение выборки

Увеличим количество положительного класса в 4 раза.

In [31]:
features_train_upsample, target_train_upsample = upsample(features_train, target_train, 4)
print(features_train_upsample.shape)
print(target_train_upsample.shape)

(9669, 11)
(9669,)


In [32]:
percent_one = target_train_upsample[target_train_upsample == 1].count() / target_train_upsample.shape[0]
print(f'Величина новой выборки - {features_train_upsample.shape[0]}. '
      f'Положительный класс составляет {percent_one:.2%} всей обучающей выборки')

Величина новой выборки - 9669. Положительный класс составляет 50.59% всей обучающей выборки


In [33]:
best_model_tree_upsample = None
if TRAIN_TREE:
    best_model_tree_upsample, best_depth, best_f1_score, best_roc_auc = train_decision_tree(
        features_train_upsample, target_train_upsample, features_valid, target_valid, 30)

    print(f'Best max depth = {best_depth}')
    print(f'Best f1 score = {best_f1_score:.4}')
    print(f'Best roc-auc = {best_roc_auc:.4}')

Best max depth = 6
Best f1 score = 0.5875
Best roc-auc = 0.8348


Best max depth = 6  
Best f1 score = 0.5875  
Best roc-auc = 0.8348  

Исследуем модель случайного леса

In [34]:
%%time
best_model_rand_forest_upsample = None
if TRAIN_FOREST:
    best_model_rand_forest_upsample, best_depth, best_n_estimators, best_f1_score, best_roc_auc = train_random_forest(
        features_train_upsample, target_train_upsample, features_valid, target_valid, 30, 155)

    print(f'Best max depth = {best_depth}')
    print(f'Best n_estimators = {best_n_estimators}')    
    print(f'Best f1 score = {best_f1_score:.4}')
    print(f'Best roc-auc = {best_roc_auc:.4}')

Best max depth = 15
Best n_estimators = 95
Best f1 score = 0.6497
Best roc-auc = 0.8541
CPU times: user 11min 6s, sys: 3.14 s, total: 11min 9s
Wall time: 11min 10s


Best max depth = 15  
Best n_estimators = 95  
Best f1 score = 0.6497  
Best roc-auc = 0.8541

Исследуем модель логистической регрессии

In [35]:
model_log_regr_upsample = None
if TRAIN_LOG_REGRESSION:
    model_log_regr_upsample, best_c, best_f1_score, best_roc_auc = train_log_regression(
        features_train_upsample, target_train_upsample, features_valid, target_valid)
    print(f'f1 score = {best_f1_score:.4}')
    print(f'roc-auc = {best_roc_auc:.4}')
    print(f'Best C = {best_c:.2}')

f1 score = 0.4843
roc-auc = 0.7604
Best C = 0.4


f1 score = 0.4843  
roc-auc = 0.7604  
Best C = 0.4

**Вывод:** Использование метода upsampling позволило добиться баланса в обучающей выборке. Мы увеличили количество положительных классов в 4 раза. В итоге положительные классы стали составлять в обучающей выборке `50.59%`. Обучение моделей на новой обучающей выборке позволило улучшить качестов моделей.  
* Модель решающего дерева показала лучший результат при максимальной глубине равной 6. Значение метрики F1 на валидационной выборке выросло до `0.5875` . Площадь под ROC кривой для данной модели составила `0.8348`, что говорит о достаточно хорошей разделяющей способности данной модели.  
* Модель случайного леса показала лучший результат при максимальной глубине равной 15 и количестве решателей равным 95. Значение метрики F1 на валидационной выборке выросло до `0.6497` . Площадь под ROC кривой для данной модели составила `0.8541`, что говорит о хорошей разделяющей способности данной модели. Это лучший результат из всех трех моделей.  
* Модель логистической регрессии, обученная на 100 итерациях, на валидационной выборке показала значение метрики F1 выросло до `0.4843` . Площадь под ROC кривой для данной модели составила `0.7604`. Разделяющая способность данной модели худшая из всех трех.  

Таким образом, лучший результат показала модель случайного леса и имеет лучшую разделяющую способнось.

#### Уменьшение выборки

Оставим в выборке 25% отрицательного класса

In [36]:
features_train_downsample, target_train_downsample = downsample(features_train, target_train, .25)
print(features_train_downsample.shape)
print(target_train_downsample.shape)

(2417, 11)
(2417,)


In [37]:
percent_one = target_train_downsample[target_train_downsample == 1].count() / target_train_downsample.shape[0]
print(f'Величина новой выборки - {features_train_downsample.shape[0]}. '
      f'Положительный класс составляет {percent_one:.2%} всей обучающей выборки')

Величина новой выборки - 2417. Положительный класс составляет 50.60% всей обучающей выборки


In [38]:
best_model_tree_downsample = None
if TRAIN_TREE:
    best_model_tree_downsample, best_depth, best_f1_score, best_roc_auc = train_decision_tree(
        features_train_downsample, target_train_downsample, features_valid, target_valid, 30)

    print(f'Best max depth = {best_depth}')
    print(f'Best f1 score = {best_f1_score:.4}')
    print(f'Best roc-auc = {best_roc_auc:.4}')

Best max depth = 5
Best f1 score = 0.5871
Best roc-auc = 0.8114


Best max depth = 5  
Best f1 score = 0.5871  
Best roc-auc = 0.8114  

Исследуем модель случайного леса

In [39]:
%%time
best_model_rand_forest_downsample = None
if TRAIN_FOREST:
    best_model_rand_forest_downsample, best_depth, best_n_estimators, best_f1_score, best_roc_auc = train_random_forest(
        features_train_downsample, target_train_downsample, features_valid, target_valid, 30, 155)

    print(f'Best max depth = {best_depth}')
    print(f'Best n_estimators = {best_n_estimators}')    
    print(f'Best f1 score = {best_f1_score:.4}')
    print(f'Best roc-auc = {best_roc_auc:.4}')

Best max depth = 8
Best n_estimators = 50
Best f1 score = 0.6082
Best roc-auc = 0.8545
CPU times: user 4min 52s, sys: 1.12 s, total: 4min 53s
Wall time: 4min 54s


Best max depth = 8  
Best n_estimators = 50  
Best f1 score = 0.6082  
Best roc-auc = 0.8545

Исследуем модель логистической регрессии

In [40]:
model_log_regr_downsample = None
if TRAIN_LOG_REGRESSION:
    model_log_regr_downsample, best_c, best_f1_score, best_roc_auc = train_log_regression(
        features_train_downsample, target_train_downsample, features_valid, target_valid)
    print(f'f1 score = {best_f1_score:.4}')
    print(f'roc-auc = {best_roc_auc:.4}')
    print(f'Best C = {best_c:.2}')

f1 score = 0.4781
roc-auc = 0.7597
Best C = 0.4


f1 score = 0.4781  
roc-auc = 0.7597  
Best C = 0.4

**Вывод:** Использование метода downsampling позволило добиться баланса в обучающей выборке. Мы оставили в выборке 25% отрицательных классов. В итоге положительные классы стали составлять в обучающей выборке `50.6%`. Обучение моделей на новой обучающей выборке позволило улучшить качестов моделей.  
* Модель решающего дерева показала лучший результат при максимальной глубине равной 5. Значение метрики F1 на валидационной выборке выросло до `0.5871` . Площадь под ROC кривой для данной модели составила `0.8114`, что говорит о достаточно хорошей разделяющей способности данной модели.  
* Модель случайного леса показала лучший результат при максимальной глубине равной 8 и количестве решателей равным 50. Значение метрики F1 на валидационной выборке выросло до `0.6082` . Площадь под ROC кривой для данной модели составила `0.8545`, что говорит о хорошей разделяющей способности данной модели. Это лучший результат из всех трех моделей.  
* Модель логистической регрессии, обученная на 100 итерациях, на валидационной выборке показала значение метрики F1 выросло до `0.4781` . Площадь под ROC кривой для данной модели составила `0.7597`. Разделяющая способность данной модели худшая из всех трех.  

Таким образом, лучший результат показала модель случайного леса и имеет лучшую разделяющую способнось.  
Однако результы, показанные моделями при обучении на выборке, полученнной методом downsampling, оказались хуже, чем при использовании метода upsampling. Скорее всего, это связано с тем, что использовании метода downsampling у нас оказалось значительно меньше данных в тренировочной выборке (9669 при upsampling и 2417 downsampling)

## Тестирование модели

Проверим лучшую модель - модель случайного леса с максимальной глубиной равной 15 и количестве решателей равным 95 на тестовой выборке.

In [44]:
if not TRAIN_FOREST:
    best_model_rand_forest = RandomForestClassifier(max_depth=15, n_estimators=95, random_state=RANDOM_STATE)
    best_model_rand_forest.fit(features_train_upsample, target_train_upsample)
else:
    best_model_rand_forest = best_model_rand_forest_upsample
predicted = best_model_rand_forest.predict(features_test)
f1, roc_auc = get_score(best_model_rand_forest, features_test, target_test)
print(f'F1 = {f1:.4}')
print(f'roc_auc = {roc_auc:.4}')

F1 = 0.6028
roc_auc = 0.8536


Таким образом на тестовой выборке модель имеет значение метрики F1 равно `0.6028`, что удовлетворяет поставленной задаче

## Общий вывод

**Общий вывод:** в ходе работы были исследованы исторические данные о поведении клиентов и расторжении договоров с банком. На первоб этапе работы была проведена предобработка данных (устранены все пропуски, закодированы категориальный признаки и нормированы количественные). Стоит отметить, что в данных содержится дисбаланс классов (в датасете преобладают данные с классом `0`). Датасет был разделен на 3 выборки:
* Тренировочная - 60%  
* Валидационная - 20%  
* Тестовая - 20%  

На втором этапе работы на данных с дисбалансом классов было обучено 3 модели:
* Модель решающего дерева. Показала лучший результат при максимальной глубине равной 2. Значение метрики F1 на валидационной выборке составило `0.5007` .  
* Модель случайного дерева. Показала лучший результат при максимальной глубине равной 22 и количестве решателей равным 45. Значение метрики F1 на валидационной выборке составило `0.6012` .  
* Модель логистической регрессии, обученная на 100 итерациях, на валидационной выборке показала значение метрики F1 равное `0.3201` .  

На третьем этапе указанный выше модели обучались на тренировочной выборке, на которой был устранен дисбаланс классов. Уравновешивание выборки выполнялось двумя способами.
1. Взвешивание классов. При данном методе модели показали следующие результаты:
    * Модель решающего дерева показала лучший результат при максимальной глубине равной 6. Значение метрики F1 на валидационной выборке выросло до `0.5875` . Площадь под ROC кривой для данной модели составила `0.8348`, что говорит о достаточно хорошей разделяющей способности данной модели.  
    * Модель случайного леса показала лучший результат при максимальной глубине равной 10 и количестве решателей равным 135. Значение метрики F1 на валидационной выборке выросло до `0.6434` . Площадь под ROC кривой для данной модели составила `0.8579`, что говорит о хорошей разделяющей способности данной модели. Это лучший результат из всех трех моделей.  
    * Модель логистической регрессии, обученная на 100 итерациях, на валидационной выборке показала значение метрики F1 выросло до `0.481` . Площадь под ROC кривой для данной модели составила `0.7602`. Разделяющая способность данной модели худшая из всех трех.  
    
Таким образом, лучший результат показала модель случайного леса и имеет лучшую разделяющую способнось.

2. С помощью увеличения выборки. Положительный класс был увеличен в 4 раза. В результате удалось добиться того, что положительный класс составлял `50.59%` обучающей выборки. При данном методе модели показали следующие результаты: 
    * Модель решающего дерева показала лучший результат при максимальной глубине равной 6. Значение метрики F1 на валидационной выборке выросло до `0.5875` . Площадь под ROC кривой для данной модели составила `0.8346`, что говорит о достаточно хорошей разделяющей способности данной модели.  
    * Модель случайного леса показала лучший результат при максимальной глубине равной 15 и количестве решателей равным 95. Значение метрики F1 на валидационной выборке выросло до `0.6497` . Площадь под ROC кривой для данной модели составила `0.8541`, что говорит о хорошей разделяющей способности данной модели. Это лучший результат из всех трех моделей.  
    * Модель логистической регрессии, обученная на 100 итерациях, на валидационной выборке показала значение метрики F1 выросло до `0.4843` . Площадь под ROC кривой для данной модели составила `0.7604`. Разделяющая способность данной модели худшая из всех трех.  
    
Таким образом, лучший результат опять показала модель случайного леса и имеет лучшую разделяющую способнось.

3. С помощью уменьшения выборки. В выборке было осавлено 25% отрицательных классов. В результате удалось добиться того, что положительный класс составлял `50.6%` обучающей выборки. При данном методе модели показали следующие результаты: 
    * Модель решающего дерева показала лучший результат при максимальной глубине равной 5. Значение метрики F1 на валидационной выборке выросло до `0.5871` . Площадь под ROC кривой для данной модели составила `0.8114`, что говорит о достаточно хорошей разделяющей способности данной модели.  
    * Модель случайного леса показала лучший результат при максимальной глубине равной 8 и количестве решателей равным 50. Значение метрики F1 на валидационной выборке выросло до `0.6082` . Площадь под ROC кривой для данной модели составила `0.8545`, что говорит о хорошей разделяющей способности данной модели. Это лучший результат из всех трех моделей.  
    * Модель логистической регрессии, обученная на 100 итерациях, на валидационной выборке показала значение метрики F1 выросло до `0.4781` . Площадь под ROC кривой для данной модели составила `0.7597`. Разделяющая способность данной модели худшая из всех трех.  
    
Таким образом, лучший результат опять показала модель случайного леса и имеет лучшую разделяющую способнось.
  
В результате экспериментов был сделан вывод, что лучшей является модель случайного леса с максимальной глубиной 15 и количеством решателей равным 95, обучнная на сбалансированной выборке, полученной с помощью увеличения выборки. Именно данная модель была проверена на тестовой выборке.
  
На четвертом этапе лучшая модель была проверена на тестовой выборке. Значение метрики F1 оказалось равным 0.6028, что удовлетворяет поставленной задаче.    
  
Таким образом, рекомендуется использовать модель случайного леса с максимальной глубиной 15 и количеством решателей равным 95.